<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20Cohere_Dataset_For_Cohere_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
chemicals = True
diseases = True

if chemicals and diseases: output = "both"
elif chemicals: output = "chemical"
elif diseases: output = "disease"

In [ ]:
!wget https://raw.githubusercontent.com/towardsai/rag-ebook-files/main/bc5cdr.json

In [ ]:
with open('bc5cdr.json') as json_file:
    data = json.load(json_file)

In [ ]:
train = 0
test = 0
develop = 0
for item in data:
  if item['dataset_type'] == "train": train+=1
  if item['dataset_type'] == "test": test+=1
  if item['dataset_type'] == "develop": develop+=1

print(train, test, develop)

500 500 500


# Get Diseases/Chemicals

In [ ]:
instruction = "The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the {} mentioned.\n\n"
outstruction = "\n\nList of extracted {}:\n"

In [ ]:
the_list = []
for item in data:
  chems = []
  dis = []

  if item['dataset_type'] == "test": continue;

  for ent in item['passages'][1]['entities']:
    if ent['type'] == "Chemical":
      if ent['text'][0] not in chems:
        chems.append( ent['text'][0] )

    if ent['type'] == "Disease":
      if ent['text'][0] not in dis:
        dis.append( ent['text'][0] )

  if chemicals: the_list.append({'prompt': instruction.format("chemicals") + item['passages'][1]['text'] + outstruction.format("chemicals"), 'completion': "- "+ "\n- ".join(chems)})
  if diseases: the_list.append({'prompt': instruction.format("diseases") + item['passages'][1]['text'] + outstruction.format("diseases"), 'completion': "- "+ "\n- ".join(dis)})

# Get Relationships

In [ ]:
instruction = "The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the influences between the chemicals and diseases mentioned.\n\n"
outstruction = "\n\nList of extracted influences:\n"

In [ ]:
the_list_rel = []
for item in data:
  if item['dataset_type'] == "test": continue;

  the_relations = []
  for rel in item['passages'][1]['relations']:
    chem_found = False
    dis_found = False
    chem = None
    dis = None
    for ent in item['passages'][0]['entities'] + item['passages'][1]['entities']:
      if len( ent['normalized'] ):
        if rel['arg1_id'] == ent['normalized'][0]['db_id'] and not chem_found: # Chemical
          chem = ent['text'][0]
          chem_found = True

        if rel['arg2_id'] == ent['normalized'][0]['db_id'] and not dis_found: # Disease
          dis_found = True
          dis = ent['text'][0]

    the_relations.append( f"- Chemical {chem} influences disease {dis}" )

  the_list_rel.append( {"prompt": instruction + item['passages'][1]['text'] + outstruction, "completion": "\n".join(the_relations)} )

In [ ]:
the_list = the_list + the_list_rel

In [ ]:
len( the_list )

3000

In [ ]:
# Writing to sample.json
with open(f"{output}_rel_instruct_all.jsonl", "w") as outfile:
  for item in the_list:
    outfile.write(json.dumps(item) + "\n")